In [1]:
#!pip install tensorflow-gpu==2.0.0-alpha

In [18]:
import tensorflow as tf
import numpy as np
import unicodedata
import re
import time
# https://colab.research.google.com/drive/1gBGBJUkPoAcQTWK6SaV9k_xBbSeaPt3M#scrollTo=tyXdcoI3BTz6
# https://colab.research.google.com/drive/1YhN8ZCZhrv18Hw0a_yIkuZ5tTh4EZDuG#scrollTo=DZgHaIdIwB_n
    
# English to bangali translation sample dataset
# https://github.com/ronypy/English-to-Bangla-machine-translation/blob/master/data/convertcsv_English_to_Bangla_Parallel_Corpus.json


In [19]:

# raw_data = (
#     ('What a ridiculous concept!', 'Quel concept ridicule !'),
#     ('Your idea is not entirely crazy.', "Votre idée n'est pas complètement folle."),
#     ("A man's worth lies in what he is.", "La valeur d'un homme réside dans ce qu'il est."),
#     ('What he did is very wrong.', "Ce qu'il a fait est très mal."),
#     ("All three of you need to do that.", "Vous avez besoin de faire cela, tous les trois."),
#     ("Are you giving me another chance?", "Me donnez-vous une autre chance ?"),
#     ("Both Tom and Mary work as models.", "Tom et Mary travaillent tous les deux comme mannequins."),
#     ("Can I have a few minutes, please?", "Puis-je avoir quelques minutes, je vous prie ?"),
#     ("Could you close the door, please?", "Pourriez-vous fermer la porte, s'il vous plaît ?"),
#     ("Did you plant pumpkins this year?", "Cette année, avez-vous planté des citrouilles ?"),
#     ("Do you ever study in the library?", "Est-ce que vous étudiez à la bibliothèque des fois ?"),
#     ("Don't be deceived by appearances.", "Ne vous laissez pas abuser par les apparences."),
#     ("Excuse me. Can you speak English?", "Je vous prie de m'excuser ! Savez-vous parler anglais ?"),
#     ("Few people know the true meaning.", "Peu de gens savent ce que cela veut réellement dire."),
#     ("Germany produced many scientists.", "L'Allemagne a produit beaucoup de scientifiques."),
#     ("Guess whose birthday it is today.", "Devine de qui c'est l'anniversaire, aujourd'hui !"),
#     ("He acted like he owned the place.", "Il s'est comporté comme s'il possédait l'endroit."),
#     ("Honesty will pay in the long run.", "L'honnêteté paye à la longue."),
#     ("How do we know this isn't a trap?", "Comment savez-vous qu'il ne s'agit pas d'un piège ?"),
#     ("I can't believe you're giving up.", "Je n'arrive pas à croire que vous abandonniez."),
# )


In [20]:
raw_data = (
(
    "A construction firm, China Major Bridge Engineering Company Ltd has been selected to construct the long aspired bridge on the river Padma.",
    "পদ্মা নদীতে দীর্ঘ আসন্ন সেতু নির্মাণের জন্য চায়না মেজর ব্রিজ ইঞ্জিনিয়ারিং কোম্পানী লিমিটেড নামক একটি কোম্পানী নির্বাচিত হয়েছে ।"
),
(
    "A number of universities are there in Bangladesh. ",
    " বাংলাদেশে বহুসংখ্যক বিশ্ববিদ্যালয় আছে। "
),
(
    "According to BBC news, Taiwanese officials said the emergency rescue operation began immediately after the earthquake. ",
    "বিবিসির খবরে বলা হয়েছে, তাইওয়ানের সরকারি কর্মকর্তারা জানিয়েছেন, ভূমিকম্পের পরপরই জরুরি উদ্ধারকাজ চালানো শুরু হয়। "
),
  (
    "Activities for tourists include angling, water skiing, river cruising, hiking, rowing, yachting and sea bathing. ",
    "পর্যটকদের জন্য ক্রিয়াকলাপগুলি আংক্রিং, ওয়াটার স্কিইং, নদী ক্রুজিং, হাইকিং, রোইং, ইয়টটিং এবং সাইড স্নান।"
  ),
  (
    "After all, if mature democracies are designed to ensure that the interests of the few do not undermine the wellbeing of the many, then how did we allow economic and political institutions to concentrate wealth to such an extent?",
    "সর্বোপরি, যদি পরিণত গণতন্ত্র তৈরিতে কিছু স্বার্থ নিশ্চিত করার জন্য অনেকের স্বার্থ নষ্ট  না হয়, তাহলে কিভাবে আমরা অর্থনৈতিক ও রাজনৈতিক প্রতিষ্ঠানগুলিকে এত পরিমাণে সম্পদ কেন্দ্রীভূত করার অনুমতি দিয়েছি?"
  ),
  (
    "All Canadians can get medical services at a reasonable price. ",
    "সকল কানাডিয়ানরা যুক্তিসঙ্গত খরচে চিকিৎসা সেবা পেতে পারে। "
  ),
  (
    "And then a single star burst into nuclear flame, sundering the void. ",
    "এবং তারপর একটি তারকা বিস্ফোরিত হয় পারমাণবিক শিখাায়, শুন্যকে ভাগ করে। "
  ),
  (
    "And, I like to laugh.",
    "এবং, আমি হাসতে চাই।"
  ),
  (
    "And, just to add a bit of spice to the announcement, it's possible that they might have discovered dark matter, a hypothesized substance that has eluded discovery for decades.",
    "এবং, ঘোষণায় মশলা যোগ করি, এটা সম্ভব যে তারা হয়তো কৃষ্ণ বস্তু আবিষ্কার করতে পেরেছেন, যে পদার্থর আবিস্কারের অনুমান কয়েক দশক ধরে এড়ানো হয়েছে।"
  ),
  (
    "As a result, every user will get extra security. ",
    "ফলে প্রত্যেক ব্যবহারকারী পাবেন বাড়তি নিরাপত্তা |"
  ),
  (
    "At office time traffic jam is intolerable. ",
    "অফিস সময় ট্রাফিক জ্যাম অসহনীয় হয়। "
  ),
  (
    "At that time, more than 150 people were rescued from hotels and residential areas in Hualian city. ",
    "এ সময় হুয়ালিয়েন শহরে হোটেল ও আবাসিক এলাকা থেকে দেড় শতাধিক মানুষকে উদ্ধার করা হয়। "
  ),
  (
    "Bangladesh and Morocco have signed an agreement on avoidance of double taxation and a protocol on foreign office consultations. ",
    "বাংলাদেশ ও মরোক্কো দ্বৈত কর এড়াতে একটি চুক্তি এবং বিদেশী অফিসের পরামর্শের জন্য একটি আচরণ বিধি স্বাক্ষর করেছে । "
  ),
  (
    "Bangladesh felt the absence of Shakib very well as they lost the Test series 1-0 with the Lions dominating the Tigers in the 2nd Test match. ",
    "বাংলাদেশ খুব ভালো ভাবে সাকিবের অনুপস্থিতি বুঝতে পেরেছিল কেননা তারা টেস্ট সিরিজ ১-০ ব্যবধানে হেরেছিল সাথে সিংহরা  দ্বিতীয় টেস্ট ম্যাচে বাঘদের উপর প্রভাব বিস্তার করেছিল। "
  ),
  (
    "Bangladesh has already promoted to the status of Low-middle income country. ",
    "ইতোমধ্যে বাংলাদেশ নিম্ন মধ্যম আয়ের দেশে উন্নীত হয়েছে। "
  ),
  (
    "Bangladesh has planned to launch a Communication & Broadcasting Satellite consisting 24 Ku and 16 C-band transponders. ",
    "বাংলাদেশ ২৪ক এবং ১৬সি ব্র্যান্ড ট্রান্সপন্ডার গুলির সমন্বয় একটি কমিউনিকেশন এবং ব্রডকাস্টিং স্যাটেলাইট চালু করার পরিকল্পনা নিয়েছে। "
  ),
  (
    "Bangladesh ranked 179th among 180 countries.",
    "এতে বাংলাদেশ ১৮০টি দেশের মধ্যে ১৭৯তম অবস্থান পেয়েছে।"
  ),
  (
    "Bangladesh's position in the index released earlier was 173 of 180 countries. ",
    "এর আগে প্রকাশিত ২০১৬ সালের সূচকে বাংলাদেশের অবস্থান ছিল ১৮০ দেশের মধ্যে ১৭৩।  "
  ),
  (
    "Bangladesh's tourist attractions include historical and monuments, resorts, beaches, picnic spots, forestsand tribal people, wildlife of various species. ",
    "বাংলাদেশ এর পর্যটন আকর্ষণসমূহ ঐতিহাসিক এবং স্মারক, রিসর্ট, সৈকত, পিকনিক স্পট, বন এবং উপজাতীয় মানুষ, বিভিন্ন প্রজাতির বন্যজীবন অন্তর্ভুক্ত।"
  ),
  (
    "Bats are the longest-lived mammals relative to body size, and a species called the greater mouse-eared bat lives especially long. ",
    "বাদুড় দেহাকৃতির তুলনায় দীর্ঘজীবী স্তন্যপায়ী এবং এর একটি প্রজাতি যার নাম ইঁদুরের মত বড় কানঅলা বাদুড় বিশেষভাবে দীর্ঘদিন বাঁচে। "
  ),
  (
    "BCB doctor Debashish Chowdhury doesn’t want to risk Shakib and he thinks he should play only when he will become fully fit.",
    "বিসিবি এর ডাক্তার দেবাশিষ চৌধুরী সাকিবকে নিয়ে ঝুঁকি নিতে চায় না এবং সে মনে করে তার শুধু তখনই খেলা উচিৎ যখন সে পুরোপুরি সুস্থ হবে।"
  ),
  (
    "Bengalis mounted a spectacular resistance and snatched victory on Dec 16 after nine months of Liberation War.",
    "বাঙালিরা অসামান্য প্রতিরোধ গড়ে তোলে এবং ১৬ ই ডিসেম্বর  বিজয় ছিনিয়ে নেয় ৯ মুক্তিযুদ্ধের পরে।"
  ),
  (
    "Bogra district was a part of the ancient Pundravardhana territory and the ruins of Mahasthangarh.",
    "বগুড়া জেলা প্রাচীন পুন্ড্রবর্ধন অঞ্চল এবং মহাস্থানগড়ের ধ্বংসাবশেষের অংশ ছিল।"
  ),
  (
    "Bogra is a northern district of Bangladesh, in the Rajshahi division. ",
    "বগুড়া হলো রাজশাহী বিভাগে বাংলাদেশ এর একটি উত্তর জেলা। "
  ),
  (
    "Bogra is an industrial city where many small and mid-sized industries are sited. ",
    "বগুড়া হলো একটি শিল্প নগরী যেখানে অনেক ছোট ও মাঝারি আকারের শিল্প প্রতিষ্ঠান অবস্থিত। "
  ),
  (
    "Both sides agreed to strengthen bilateral cooperation in economic development, trade and investment, agriculture and tourism cooperation. ",
    "উভয় পক্ষ অর্থনৈতিক উন্নয়ন, বাণিজ্য ও বিনিয়োগ, কৃষি ও পর্যটন সহযোগিতার ক্ষেত্রে দ্বিপাক্ষিক সহযোগিতা জোরদার করতে সম্মত হয়েছে। "
  ),
  (
    "But it often refers also to its solid state (ice) or its gaseous state (steam or water vapor).",
    "কিন্তু এটি প্রায়ই তার কঠিন অবস্থা (বরফ) বা তার বায়বীয় অবস্থা (বাষ্প বা পানির বাষ্প) উল্লেখ করে।"
  ),
  (
    "But major indexes posted their worst week of losses since early February as Trump's threat to impose import tariffs on steel and aluminium rattled investors.",
    "কিন্তু প্রধান সূচকগুলি জানায় তারা গত ফেব্রুয়ারী থেকে তাদের সবচেয়ে লোকসানের সপ্তাহ পার করলো, যেহেতু, ট্রাম্প এর ইস্পাত ও অ্যালুমিনিয়ামে আমদানি শুল্ক আরোপের হুমকি বিনিয়োগকারীদের আতঙ্কিত করে তোলে। "
  ),
  (
    "But they can not be resourceful without support from the society they live in. ",
    "কিন্তু তারা যে সমাজে মধইয়্য বসবাস করে সে সমাজের সমর্থন ছাড়া সম্পদশালী হতে পারে না। "
  ),
  (
    "Canada is a very beautiful country. ",
    "কানাডা খুব সুন্দর একটি  দেশ। "
  ),
  (
    "Canada, the biggest supplier of steel and aluminium to the United States, said it would retaliate if hit by US tariffs. ",
    "কানাডা, মার্কিন যুক্তরাষ্ট্রে ইস্পাত এবং অ্যালুমিনিয়ামের বৃহত্তম সরবরাহকারী, বলে, তারা প্রতিশোধ নিবে  যদি মার্কিন ট্যারিফ দ্বারা আঘাত প্রাপ্ত হয়।  "
  ),
  (
    "Canadian cities have many parks and lots of space for people to live. ",
    "কানাডিয়ান শহরগুলিতে অনেক উদ্যান এবং মানুষের বসবাসের অনেক স্থান আছে। "
  ),
  (
    "Children get good environment to grow up. ",
    "শিশুরা বেড়ে উঠার সুন্দর পরিবেশ পায়।  "
  ),
  (
    "Chittagong is a major coastal seaport city and financial centre in southeastern Bangladesh. ",
    "চট্টগ্রাম দক্ষিণপূর্ব বাংলাদেশের একটি প্রধান উপকূলবর্তী সমুদ্র বন্দর শহর এবং আর্থিক কেন্দ্র। "
  ),
  (
    "Codenamed ‘Operation Searchlight’, it carried out genocide in the first hours of that night in Dhaka. ",
    "ঢাকায় রাতের প্রথম প্রহরে গণহত্যা চালায়, যার সাংকেতিক  নাম 'অপারেশন সার্চলাইট'।"
  ),
  (
    "During the final of the tri-nation series against Sri Lanka, all-rounder Shakib Al Hasan injured his finger while fielding and it didn’t only ruled him out from that match, but also for the two match Test series.",
    "শ্রীলঙ্কার বিপক্ষে ত্রিদেশীয় সিরিজের ফাইনাল চলাকালীন, চৌকস খেলোয়াড় সাকিব আল হাসান তার আঙুলে চোট পায় যখন সে ফিল্ডিং করছিল এবং এটা শুধু তাকে ঐ ম্যাচ থেকেই বাতিল করে নি, দুই ম্যাচের টেস্ট সিরিজ থেকেও করেছে।"
  ),
  (
    "Earthquake felt on the island last night at 6.4 magnitude. ",
    "গতকাল  রাতে দ্বীপটিতে ৬ দশমিক ৪ মাত্রার এ ভূমিকম্প অনুভূত হয়। "
  ),
  (
    "Education is the backbone of a nation. ",
    "শিক্ষাই জাতির মেরুদন্ড।"
  ),
  (
    "Educational institutes are called the nation-building workshop. ",
    "শিক্ষা প্রতিষ্ঠানগুলোকে জাতি গড়ার কারিগর বলা হয়।"
  ),
  (
    "Eighteen of them are bats, some living more than four decades. ",
    "তাদের মধ্যে আঠারো হচ্ছে বাদুড় , যাদের মধ্যে কিছু চার দশকের চেয়ে বেশী বাঁচে। "
  ),
  (
    "Environmental Performance Index 2018 (EPI) recently published on the overall status of the environment. ",
    "পরিবেশের সার্বিক অবস্থা নিয়ে সম্প্রতি প্রকাশিত হলো এনভায়রনমেন্টাল পারফরমেন্স ইনডেক্স ২০১৮ (ইপিআই)। "
  ),
  (
    "Especially the affected areas have been severely damaged. ",
    "বিশেষ করে উপদ্রুত অঞ্চলের রাস্তাঘাট ব্যাপক ক্ষতিগ্রস্ত হয়েছে।"
  ),
  (
    "Every two years, the global index was released.",
    "প্রতি দুই বছর পর বৈশ্বিক এ সূচক প্রকাশ করা হয়।"
  ),
  (
    "Finally, Canada's cities are clean and efficiently managed. ",
    "অবশেষে, কানাডার শহরগুলো পরিচ্ছন্ন এবং দক্ষভাবে পরিচালিত হয়। "
  ),
  (
    "First, Canada has an excellent health care system. ",
    "প্রথমত, কানাডার উন্নতমানের স্বাস্থ্য সেবা ব্যবস্থা আছে। "
  ),
  (
    "Foreign Minister Abul Hassan Mahmood Ali and his Moroccan counterpart Nasser Bourita signed the two instruments at the Ministry of Foreign Affairs in Rabat on Wednesday. ",
    "পররাষ্ট্রমন্ত্রী আবুল হাসান মাহমুদ আলী এবং তার মরোক্কোর অনুরূপ পক্ষ নাসের বুরতা বুধবার রাবাতে পররাষ্ট্র মন্ত্রণালয়ের দুটি দলিলে স্বাক্ষর করেন।"
  ),
  (
    "He went on to conservatively point out that both claims are extraordinary and require extraordinary evidence.",
    "তিনি দৃঢ়তার সাথে বলেন যে উভয় দাবি অসাধারণ এবং লক্ষণীয় প্রমাণ প্রয়োজন।"
  ),
  (
    "Here it goes.",
    "এখানে এটা যায়।"
  ),
  (
    "Hey, this is my homepage, so I have to say something about myself. ",
    "হেই, এটা আমার হোমপেজ, তাই আমার সম্পর্কে কিছু বলতে হবে। "
  ),
  (
    "However, these requirements shall be finalized during the preparation and finalization of tender document for launching the satellite.",
    "যাইহোক এই প্রয়োজনীয়তাগুলি উপগ্রহের জন্য লেনদেন এর প্রস্তুতি এবং চূড়ান্ত করনের সময় চূড়ান্ত করা হবে।"
  ),
  (
    "I am a person who is positive about every aspect of life. ",
    "আমি একজন ব্যক্তি যিনি জীবনের প্রতিটি দিক সম্পর্কে ইতিবাচক। "
  ),
  (
    "I hope that my impression about myself and your impression about me are not so different. ",
    "আমি আশা করি আমার সম্পর্কে আমার ধারণা এবং আমার সম্পর্কে আপনার ধারনা খুব বেশি ভিন্ন নয়। "
  ),
  (
    "I like delicious food and comfortable shoes. ",
    "আমি সুস্বাদু খাদ্য এবং আরামদায়ক জুতা পছন্দ করি। "
  ),
  (
    "I like flowers in spring, rain in summer, leafs in autumn and snow in winter. ",
    "আমি বসন্তে ফুল পছন্দ করি, গ্রীষ্মে বৃষ্টি, শরত্কালে পাতা এবং শীতকালে তুষারপাত। "
  ),
  (
    "I like good books and romantic movies.",
    "আমি ভাল বই এবং রোমান্টিক সিনেমা পছন্দ করি। "
  ),
  (
    "I like the land and the nature, I like people. ",
    "আমি ভূমি এবং প্রকৃতি পছন্দ করি, আমি মানুষ পছন্দ করি। "
  ),
  (
    "I like to look at the clouds in the sky with a blank mind. ",
    "আমি একটি খালি মন দিয়ে আকাশে মেঘ দেখতে পছন্দ করি।"
  ),
  (
    "I like to read, I like to write, I like to think, I like to dream, I like to talk, I like to listen. ",
    "আমি পড়তে পছন্দ করি, আমি লিখতে পছন্দ করি; আমি চিন্তা করতে পছন্দ করি, আমি স্বপ্ন দেখতে পছন্দ করি; আমি কথা বলতে পছন্দ করি, আমি শুনতে পছন্দ করি। "
  ),
  (
    "I like to see the sunrise in the morning, I like to see the moonlight at night. ",
    "আমি সকালে সূর্যোদয় দেখতে পছন্দ করি, আমি রাতে চাঁদের আলো দেখতে পছন্দ করি। "
  ),
  (
    "I like to sleep early, I like to get up late, I like to be alone, I like to be surrounded by people. ",
    "আমি তাড়াতাড়ি ঘুমাতে পছন্দ করি,  আমি দেরি করে উঠতে চাই, আমি একা হতে পছন্দ করি, চাই, আমি মানুষ দ্বারা বেষ্টিত হতে পছন্দ করি।"
  ),
  (
    "If you have problems with excess oil, fungal or bacterial infestation, it is likely that your scalp and hair are too alkaline. ",
    "আপনার যদি অতিরিক্ত তেল, ছাত্রাক অথবা ব্যাকটেরিয়া পরিব্যাপ্তি জনিত সমস্যা থাকে, এটা সম্ভাব্য যে আপনার স্কাল্প ও চুল অতিরিক্ত ক্ষারীয়। "
  ),
  (
    "In 2016, a Saudi woman was detained for removing her abaya on a main street in the capital of Riyadh. ",
    "২০১৬ সালে, রিয়াদের রাস্তায় এক নারী তাঁর আবা খুলে ফেললে পুলিশ তাঁকে গ্রেপ্তার করে। "
  ),
  (
    "In case of Bangladesh, the state is committed to the provision of basic necessities including adoption of uniform, mass-oriented, free and compulsory education through which an equitable society can be created so that all kinds of exploitation can be removed.   ",
    "বাংলাদেশের ক্ষেত্রে, রাষ্ট্র মৌলিক প্রয়োজনগুলো যোগাতে প্রতিশ্রুতিবদ্ধ, যেখানে অন্তভুক্ত অভিন্ন, গণ-মুখী, বিনামুল্যে ও বাধ্যতামূলক শিক্ষা গ্রহন, যার মাধ্যমে একটি ন্যায়সঙ্গত সমাজ সৃষ্টি করতে পারে যাতে সব ধরণের শোষণ অপসারিত করতে পারে।"
  ),
  (
    "In proportion to our population roads have not increased. ",
    "আমাদের জনসংখ্যার অনুপাতে সড়ক বেড়ে যায় নি। "
  ),
  (
    "In the 2014 list, Bangladesh's position was 169 among 178 countries of the world.",
    "২০১৪ সালের তালিকায় বিশ্বের ১৭৮টি দেশের মধ্যে বাংলাদেশের অবস্থান ছিল ১৬৯।"
  ),
  (
    "In the northern part, comprising the Rajshahi division, there are archaeological sites including the temple city Puthia in Rajshahi; the largest and most ancient archaeological site, Mahasthangarh in Bogra; the single largest Buddhist monastery, Paharpur in Naogaon; the  most ornamental terracota Hindu temple, Kantaji temple and many rajbaris.",
    "উত্তরের অংশে রাজশাহী বিভাগের অন্তর্গত, রাজশাহীতে মন্দিরের শহর পুথিসহ প্রত্নতাত্ত্বিক স্থান রয়েছে; বৃহত্তম এবং প্রাচীন প্রত্নতাত্ত্বিক সাইট,  বগুড়া মহাস্থানগড়; একক বৃহত্তম বৌদ্ধ মঠ, নওগাঁর পাহাড়পুর সবচেয়ে শোভাময় পোড়ামাটির হিন্দু মন্দির, কান্তজী মন্দির এবং অনেক রাজবাড়ী।"
  ),
  (
    "In what could well be a stunning breakthrough, a group of astronomers have announced that they have found radio signals that appear to provide evidence of the first stars to come into existence. ",
    "কি এক দুর্দান্ত সাফল্য, জ্যোতির্বিজ্ঞানীদের একটি দল ঘোষণা দিয়েছেন তারা রেডিও সংকেত খুজে পেয়েছেন যা জীবনযাত্রার প্রথম তারার। "
  ),
  (
    "Inequality has emerged as one of the most contentious topics over the last one decade. ",
    "গত এক দশকে বৈষম্য সবচেয়ে বিতর্কিত বিষয়গুলির মধ্যে একটি হিসাবে আবির্ভূত হয়েছে। "
  ),
  (
    "It has not only found center stage in bitter political debates and academic discourses, the extreme inequality that currently prevails in advanced industrial societies has raised many question regarding the degree of real influence average citizens have in shaping laws and policies. ",
    "এটা শুধু তিক্ত রাজনৈতিক বিতর্ক এবং একাডেমিক আলোচনার কেন্দ্রে স্থান পায়নি, বর্তমানে উন্নত শিল্প সমাজে বিদ্যমান প্রকট বৈষম্য নীতিমালা গঠন করার ক্ষেত্রে সাধারণ নাগরিকদের প্রকৃত প্রভাবের সম্পর্কে অনেক প্রশ্ন উত্থাপিত হয়েছে। "
  ),
  (
    "It is a significant achievement for our country. ",
    "এটি আমাদের দেশের জন্য একটি গুরুত্বপূর্ণ অর্জন। "
  ),
  (
    "It is called the gateway to North Bengal. ",
    "একে উত্তরবঙ্গের প্রবেশপথ বলা হয়। "
  ),
  (
    "It is one of the major problems of modern time. ",
    "এটা আধুনিক সময় এক প্রধান সমস্যা। "
  ),
  (
    "It was recognised in 1997 as a UNESCO World Heritage Site of Bangladesh. ",
    "এটি 1997 সালে ইউনেস্কো ওয়ার্ল্ড হেরিটেজ সাইট হিসাবে স্বীকৃত ছিল।"
  ),
  (
    "It was reported that more than 200 people were injured. ",
    "এতে দুই শতাধিক মানুষ আহত হয়েছে বলে খবর পাওয়া গেছে। "
  ),
  (
    "It will connect Louhajong, Munshiganj to Shariatpur and Madaripur, linking the south-west of the country, to northern and eastern regions. ",
    "এর মাধ্যমে লৌহজং, মুন্সিগঞ্জের সাথে শরিয়তপুর ও মাদারীপুর যুক্ত হবে, ফলে দেশের দক্ষিণ-পশ্চিম অঞ্চলের সাথে উত্তর-পূর্ব অঞ্চলের সংযোগ ঘটবে।"
  ),
  (
    "Its chemical formula is H2O, meaning that each of its molecules contains one oxygen and two hydrogen atoms that are connected by covalent bonds. ",
    "এর রাসায়নিক সূত্র H2O হয়, যার মানে প্রতিটি অণুতে একটি অক্সিজেন এবং দুটি হাইড্রোজেন পরমাণু থাকে যা যৌথ বন্ড দ্বারা সংযুক্ত। "
  ),
  (
    "Just after the end of the 2nd Test match, the Bangladesh Cricket Board (BCB) announced 15-man squad for the first T20 match.",
    "দ্বিতীয় ম্যাচ এর পরপরই, বাংলাদেশ ক্রিকেট বোর্ড(বিসিবি) ১৫ সদস্যের প্রথম টি২০ ম্যাচ এর দল ঘোষণা করেছিল।"
  ),
  (
    "just like the skin on your face, the scalp sheds dead skin cells and it can be covered in a film of hair products and silicon from years of buildup. ",
    "ঠিক আপনার মুখের ত্বকের মতোই, স্কাল্প থেকেও মৃত কোষ ঝরে এবং এটা অনেক বছর ধরে জমা  চুলের প্রসাধনীর ও সিলিকন একটি আবরণ দারা আবৃত হয়ে জেতে পারে । "
  ),
  (
    "Let me give a try to see what kind of image you have about me through my self-description. ",
    "আমার আত্ম-বিবরণীর মাধ্যমে আমার সম্পর্কে আপনাদের কি ধরনের প্রতিচ্ছবি তৈরি হয় সেটা দেখার জন্য আমাকে একটা চেষ্টা করতে দিতে দিন। "
  ),
  (
    "Library of NSTU is well furnished containing about 3500 books and around 500 local and foreign journals. ",
    "এনএসটিইউ এর গ্রন্থাগার প্রায় ৩৫০০ টি বই এবং ৫০০ টি স্থানীয় ও বিদেশী সংবাদপত্রে সুসজ্জ্বিত। "
  ),
  (
    "Local media reported that she was detained after a complaint was filed with the religious police.",
    "স্থানীয় গণমাধ্যম জানায়, অভিযোগের পর ধর্মীয় পুলিশি তাকে আটক করা হয়।"
  ),
  (
    "Local media reports said there was a hospital and hotel building damaged in the earthquake.",
    "স্থানীয় সংবাদমাধ্যমের প্রতিবেদনে বলা হয়েছে, ভূমিকম্পে ক্ষতিগ্রস্ত ভবনের মধ্যে একটি হাসপাতাল ও হোটেল আছে।"
  ),
  (
    "Located in the delta region of Padma, Meghna and Brahmaputra river basins.",
    "পদ্মা, মেঘনা ও ব্রহ্মপুত্র নদী উপত্যকার ডেল্টা অঞ্চলে অবস্থিত। "
  ),
  (
    "Maintaining the correct pH value of the hair and the scalp is essential for the overall health of hair.",
    "চুলের স্বাস্থ্য ঠিক রাখতে সঠিক pH এর  সঠিক  মান রক্ষা করা জরুরী। "
  ),
  (
    "Modern humans are the only extant members of the subtribe Hominina, a branch of the tribe Hominini belonging to the family of great apes. ",
    "আধুনিক মানুষ হল টি একমাত্র বিদ্যমান সদস্য এর টি উপজাতি হোমিনিনা, একটি শাখা টি গোত্র হোমিনিনি অন্তর্গত থেকে টি পরিবার এর মহান বানর ।"
  ),
  (
    "Most important feature of this library is online library.",
    "এই গ্রন্থাগারের একটা উল্লেখযোগ্য দিক হল এর অনলাইন গ্রন্থাগার। "
  ),
  (
    "Much of Chittagong Division is located within the ecological Indo-Burma zone on the boundary of the India Plate and Burma Plate.",
    "চট্টগ্রাম বিভাগের বেশির ভাগ অংশ ভারত প্লেট এবং বার্মা প্লেটের সীমানার উপরে অবস্থিত ইন্দো-বার্মা জোন এলাকায় অবস্থিত। "
  ),
  (
    "Natural beauty around the university is appreciable in comparison with other universities. ",
    "প্রাকৃতিক সৌন্দর্যের বিবেচনায় বিশ্ববিদ্যালয়টি অন্যান্য বিশ্ববিদ্যালয়ের তুলনায় প্রশংসনীয়। "
  ),
  (
    "Noakhali Science and Technology University (NSTU) is one of them and one of the best public universities by virtue of its subjects. ",
    "নোয়াখালী বিজ্ঞান ও প্রযুক্তি বিশ্ববিদ্যালয় (এনএসটিইউ)  তাদের মধ্যে অন্যতম এবং এর বিষয়গুলোর কারনে এটি একটি সেরা জাতীয় বিশ্ববিদ্যালয়। "
  ),
  (
    "Noted educational institutions are Shaeed Ziaur Rahman Medical College, Government Azizul Huq College, Bogra Zila School, Bogra Government Girl's High School, Kahaloo High School, Sonatola Pilot High School, Adamdighi High School, Gosaibari AA High School, Talora Altaf Ali High School, Dupchachia Pilot Girl's High School, Shibganj Pilot High School, Dupchachia Pilot High School, Nawkhila Government Primary School, Devdanga Government Primary School, Chagaldhara Government Primary School.",
    "উল্লেখযোগ্য শিক্ষা প্রতিষ্ঠানগুলি হলো শহীদ জিয়াউর রহমান মেডিকেল কলেজ, সরকারি আজিজুল হক কলেজ, বগুড়া জিলা স্কুল, বগুড়া সরকারি বালিকা উচ্চ বিদ্যালয়, কাহালু উচ্চ বিদ্যালয়, সোনাতলা পাইলট উচ্চ বিদ্যালয়, আদমদীঘি উচ্চ বিদ্যালয়, গোসাইবাড়ী এএ উচ্চ বিদ্যালয়, তালোড়া আলতাফ আলী উচ্চ বিদ্যালয়, দুপচাঁচিয়া পাইলট বালিকা উচ্চ বিদ্যালয়, শিবগঞ্জ পাইলট উচ্চ বিদ্যালয়, দুপচাঁচিয়া পাইলট উচ্চ বিদ্যালয়, নওখিলা সরকারি প্রাথমিক বিদ্যালয়, দেবডাঙ্গা সরকারি প্রাথমিক বিদ্যালয়, ছাগলধরা সরকারি প্রাথমিক বিদ্যালয়।"
  ),
  (
    "Now the country is trying to cross the border of middle income status and watching dream to become a developed country. ",
    "এখন মধ্যম আয় পেরিয়ে উন্নত দেশে উত্তরণের স্বপ্ন দেখছে। "
  ),
  (
    "Only 19 mammal species are longer-lived than humans relative to body size. ",
    "শুধু মাত্র ১৯ স্তন্যপায়ী প্রজাতি দেহাকৃতির সাপেক্ষে মানুষের তুলনায় দীর্ঘজীবী।"
  ),
  (
    "Overtaking tendency also causes traffic ham. ",
    "ওভারকিং প্রবণতা এছাড়াও ট্র্যাফিক জ্যামের কারণ। "
  ),
  (
    "Padma Bridge is the most challenging construction project in the history of Bangladesh. ",
    "বাংলাদেশের ইতিহাসে পদ্মা সেতু হতে যাচ্ছে একটি সবচেয়ে বড় চ্যালেঞ্জিং নির্মাণ প্রকল্প। "
  ),
  (
    "People are the most important resources for any nation. ",
    "যেকোন জাতির জন্য মানুষ সবচেয়ে গুরুত্বপূর্ণ সম্পদসমুহ। "
  ),
  (
    "Recent visits of prime minister are the expression of that.",
    "তারই প্রকাশ দেখা যায় প্রধানমন্ত্রীর সাম্প্রতিক সফরগুলোতে।"
  ),
  (
    "Researchers now have unlocked some of this bat's longevity secrets, with hints for fighting the effects of aging in people.",
    "বর্তমানে গবেষকগণ এমন কিছু বাদুড়ের আয়ুষ্কাল রহস্য এর সাথে মানুষের বার্ধক্য প্রভাব প্রতিরোধের সংকেত উদঘাটন করেন। "
  ),
  (
    "Saudi women have started wearing more colorful abayas in recent years.",
    "বর্তমান সময়ে সৌদি মহিলারা অধিক রঙ্গিন আবা গুলো পরা শুরু করেছে।"
  ),
  (
    "Scalp needs to be treated with focus on exfoliation and balancing pH.",
    "স্কাল্প চিকিৎসা করা উচিৎ  ভারসাম্যযুক্ত  pH ও এক্সফলিএশন উপর গুরুত্ব রেখে। "
  ),
  (
    "Scalp oil  has a pH balance of between 4.5 and 5.5. ",
    "স্কাল্প তেল (Scalp Oil) এ pH এর মান ৪.৫ থেকে ৫.৫ এর মধ্যে। "
  ),
  (
    "Scientists are working on an artificial intelligence (AI) system that can be operated without internet connection and cloud computing help.",
    "বিজ্ঞানীরা এমন একটি কৃত্রিম বুদ্ধিমত্তা (এআই) পদ্ধতি নিয়ে কাজ করছেন, যা ইন্টারনেট সংযোগ ও ক্লাউড কম্পিউটিং সাহায্য ছাড়াই পরিচালনা করা যাবে। "
  ),
  (
    "Scientists said on Wednesday that unlike in people and most other animals, in this bat species the structures called telomeres located at the end of chromosomes, thread-like strands inside a cell's nucleus that carry genes determining heredity, do not shorten with age.",
    "বিজ্ঞানীরা বুধবার বলেছিলেন যে মানুষ এবং বেশিরভাগ অন্য প্রানির চেয়ে আলাদা এই বাদুড় প্রজাতিতে একটি কাঠামো যাকে টেলমেয়ার বলে তা  ক্রোমোসোম এর শেষে অবস্থিত, সূতার মতন সুক্ষ আশ যা  কোষের নিউক্লিয়াসে বংশধারা নিরুপক জিন বহন করে, বয়সের সাথে সংকুচিত হয় না।"
  ),
  (
    "Second, Canada has a high standard of education system. ",
    "দ্বিতীয়ত, কানাডার শিক্ষা ব্যবস্থা খুব উন্নতমানের। "
  ),
  (
    "Several of these hominins used fire, occupied much of Eurasia, and gave rise to anatomically modern Homo sapiens in Africa about 315,000 years ago.",
    "কিছু অংশ হোমিনিন ব্যবহার করত আগুন, দখল করা অধিকাংশ  ইউরেশিয়া, এবং মাথাচাড়া দিয়ে ওঠেছিল উদ্ভব আধুনিক হোমো স্যাপিয়েন্স মধ্যে আফ্রিকা প্রায় 315000 বছর পূর্বে  ।"
  ),
  (
    "Several structures of the city have been broken by the earthquake and the ruins are seen spread all over. ",
    "ভূমিকম্পে শহরটির বিভিন্ন কাঠামো ভেঙে গেছে এবং চারদিকে ধ্বংসাবশেষ ছড়িয়ে-ছিটিয়ে পড়ে থাকতে দেখা গেছে। "
  ),
  (
    "Since 21 May 1992, the Sundarbans is recognized as a Ramsar Site of ecological importance.",
    "২১ মে ১৯৯২ সাল থেকে, সুন্দরবন পরিবেশগত গুরুত্ব একটি রামসার স্থান হিসেবে স্বীকৃত।"
  ),
  (
    "So, Canada is a desirable place for living.",
    "সুতরাং, কানাডা বসবাসের জন্য একটি আকাঙ্খিত স্থান।"
  ),
  (
    "Social process and education provisions together are the means of producing resourceful human beings. ",
    "সামাজিক প্রক্রিয়া ও শিক্ষা ব্যবস্থা একসঙ্গে সম্পদশালী মানবজাতী তৈরী করার মাধ্যম । "
  ),
  (
    "Sometimes it is hard to introduce yourself because you know yourself so well that you do not know where to start with. ",
    "কখনও কখনও নিজেকে পরিচয় করিয়ে দেওয়া কঠিন কারণ আপনি নিজেকে এত ভাল জানেন যে আপনি কোথা থেকে শুরু করবেন তা জানেন না। "
  ),
  (
    "Sometimes traffic jam is so heavy that it blocks half a kilometer.",
    "কখনও কখনও ট্র্যাফিক জ্যাম এতই বেশি যে এটি অর্ধ কিলোমিটার ব্লক করে।"
  ),
  (
    "Strictly speaking, water refers to the liquid state of a substance that prevails at standard ambient temperature and pressure.",
    "কঠোরভাবে বলছেন, পানি এমন একটি পদার্থের তরল অবস্থাকে নির্দেশ করে যা মান পরিমাপের তাপমাত্রা এবং চাপে প্রবল। "
  ),
  (
    "Students are taught by well‐trained teachers and are encouraged to continue studying at university. ",
    "শিক্ষার্থীদের ভাল প্রশিক্ষিত শিক্ষকদের দ্বারা শেখানো হয় এবং বিশ্ববিদ্যালয়ে পড়ালেখা চালিয়ে যেতে উৎসাহ দেয়া হয়। "
  ),
  (
    "Sundarban is the part of bangladesh and india.",
    "সুন্দরবন বাংলাদেশ এবং ভারত এর অংশ।"
  ),
  (
    "Telomeres are protective caps at the ends of chromosomes that shorten each time a cell divides. ",
    "টেলমেয়ার হল ক্রোমোসোমের শেষ প্রান্তে অবস্থিত সংরক্ষণশীল সুরক্ষIমুলক উপরিভাগ যা প্রতিবার কোষ বিভাজনের সময় সংকুচিত হয়। "
  ),
  (
    "The atmosphere of the university campus is very much convenient and favorable for study. ",
    "বিশ্ববিদ্যালয় প্রাঙ্গনের বায়ুমন্ডল পড়াশুনার জন্য অনুকূল ও উপযোগী।"
  ),
  (
    "The average literacy rate is 42.9%; male 48%, female 37.5%. ",
    "গড় সাক্ষরতার হার ৪২.৯%; পুরুষ ৪৮%, মহিলা ৩৭.৫%। "
  ),
  (
    "The causes of traffic jam are many. ",
    "ট্র্যাফিক জ্যাম অনেক কারণ আছে। "
  ),
  (
    "The city area of Borga is approximately 72.5 square kilometer ( 27.99 square miles ), having 21 wards, and has a population of around 780,000 people. ",
    "বগুড়া শহরের ক্ষেত্রফল প্রায় ৭২.৫ বর্গ কিলোমিটার (২৭.৯৯ বর্গ মাইল), যাতে রয়েছে ২১ টি ওয়ার্ড এবং এটার লোকসংখ্যা প্রায় ৭৮০,০০০ জন। "
  ),
  (
    "The city has a population of more than 2.5 million while the metropolitan area has a population of 4,009,423 at the 2011 Census, making it the second largest city in the country.",
    "শহরটির ২.৫ মিলিয়ন লোকের জনসংখ্যা রয়েছে এবং ২০১১ সালের আদমশুমারি অনুযায়ী মেট্রোপলিটন এলাকার ৪,০০৯,৪২৩ জন জনসংখ্যা রয়েছে, এটি দেশের দ্বিতীয় বৃহত্তম শহর। "
  ),
  (
    "The decision came at a recent inter-ministerial meeting headed by Liberation War Affairs Minister AKM Mozammel Huq on observing Mar 26 Independence Day and the Genocide Day. ",
    "মুক্তিযুদ্ধ বিষয়ক মন্ত্রী একেএম মোজাম্মেল হক নেতৃত্বে একটি সাম্প্রতিক আন্তঃমন্ত্রণালয় সভায় 26 শে মার্চ স্বাধীনতা দিবস এবং গণহত্যা দিবস হিসেবে পালনের সিদ্ধান্ত গৃহীত হয়। "
  ),
  (
    "The drivers are not willing to obey the traffic rules. ",
    "ড্রাইভার ট্রাফিক আইন মান্য করতে ইচ্ছুক নয়। "
  ),
  (
    "The EPI index has been jointly developed by Yale University of America and Columbia University's Center for International Earth Science Information Network. ",
    "ইপিআই সূচকটি যৌথভাবে তৈরি করেছে যুক্তরাষ্ট্রের ইয়েল ইউনিভার্সিটি ও কলাম্বিয়া ইউনিভার্সিটির সেন্টার ফর ইন্টারন্যাশনাল আর্থ সায়েন্স ইনফরমেশন নেটওয়ার্ক। "
  ),
  (
    "The European Union raised the possibility of taking countermeasures, France said the duties would be unacceptable, and China urged Trump to show restraint. ",
    "ইউরোপীয় ইউনিয়নের প্রতিব্যবস্থা গ্রহণের সম্ভাবনা উত্থাপন করে, ফ্রান্স বলেছে যে শুল্কহার গুলো অগ্রহণযোগ্য, এবং চীন সংযম দেখানোর জন্য ট্রাম্পের প্রতি আহ্বান জানায়। "
  ),
  (
    "The government has decided to go for a one-minute blackout from 9pm on the day, which was officially declared as Genocide Day last year. ",
    "সরকার গতকাল 9 টা থেকে এক মিনিট ব্ল্যাকআউটের জন্য যেতে সিদ্ধান্ত নিয়েছে, যা আনুষ্ঠানিকভাবে গত বছর গণহত্যা দিবস হিসাবে ঘোষণা করা হয়েছিল। "
  ),
  (
    "The government’s Center for International Communications did not immediately respond to requests for comment.",
    "সরকারের আন্তর্জাতিক যোগাযোগ কেন্দ্র তত্ক্ষণাৎ কোন প্রতিক্রিয়া দেখাই নি।"
  ),
  (
    "The issues that are considered to create an EPI index are: the effects of pollution on public health, air quality, water supply and sanitation, water resources, agriculture, forestry, fisheries, biodiversity and housing and weather and energy.",
    "ইপিআই সূচক তৈরিতে বিবেচ্য বিষয়গুলো হলো- জনস্বাস্থ্যের ওপর দূষণের প্রভাব, বায়ুর মান, পানি সরবরাহ ও পয়োনিষ্কাশন, পানিসম্পদ, কৃষি, বনায়ন, মত্স্যসম্পদ, জীববৈচিত্র্য ও বাসস্থান এবং আবহাওয়া ও জ্বালানি।"
  ),
  (
    "The Kingdom has seen an expansion in women’s rights recently, such as the decision passed to allow women to attend mixed public sporting events and the announcement that Saudi Arabia would grant them the right to drive.",
    "সাম্প্রতিককালে রাজ্য নারীদের অধিকারে গুলোর সম্প্রসারণ দেখেছে, যেমন নারীরা মিশ্র পাবলিক স্পোর্টিং ইভেন্টে অংশগ্রহণের অনুমতি দেওয়ার সিদ্ধান্ত নিয়েছে এবং সৌদি আরব তাদের ড্রাইভ করার অধিকার প্রদান করবে ।"
  ),
  (
    "The ministry has written to all government agencies over switching off the lights everywhere, except for key point installations, for a minute. ",
    "মন্ত্রণালয় সব সরকারী সংস্থার কাছে এক মিনিটের জন্য লাইট বন্ধ করার জন্য লিখিত আদেশ দিয়েছে, জরুরি স্থাপনা বাদে। "
  ),
  (
    "The Moroccan foreign minister discussed the possibility of technical assistance and cooperation in phosphate fertilizer industry in Bangladesh, including production of customized and cost-effective phosphate fertilizer suitable for different types of soil.",
    "মরক্কোর পররাষ্ট্র মন্ত্রী বিভিন্ন ধরণের মাটির জন্য উপযোগী বিশেষয়িত এবং সুলভ মুল্যের কার্যকর ফসফেট সার উত্পাদন সহ বাংলাদেশে ফসফেট সার শিল্পে কারিগরি সহায়তা এবং সহযোগিতার সম্ভাবনা নিয়ে আলোচনা করেন ।  "
  ),
  (
    "The new 'Deep Learning AI software' can be easily installed on mobile phones and computer chips and can be used in all the robots in the factory from the smartphone itself. ",
    "নতুন এই 'ডিপ লার্নিং এআই সফটওয়্যার' খুব সহজেই মোবাইল ফোন ও কম্পিউটার চিপে এঁটে যেতে পারে এবং স্মার্টফোন থেকে শুরু করে কারখানায় ব্যবহৃত রোবট সবক্ষেত্রেই ব্যবহার করা যাবে|  "
  ),
  (
    "The number of traffic police is insufficient. ",
    "ট্রাফিক পুলিশের সংখ্যা অপর্যাপ্ত। "
  ),
  (
    "The other is a weird African rodent called a naked mole rat.",
    "অন্য একটি অদ্ভুত আফ্রিকান তীক্ষ্ণ দন্ত বিশিষ্ট যাকে নগ্ন ছুঁচ বলে। "
  ),
  (
    "The Padma Bridge is a multipurpose road and rail bridge across the Padma River under construction in Bangladesh.",
    "পদ্মা সেতু বাংলাদেশের পদ্মা নদীর উপর নির্মাণাধীন একটি বহুমুখী সড়ক ও রেল সেতু। "
  ),
  (
    "The Primary Service Area (PSA) would be Bangladesh and neighboring countries and the Secondary Service Area (SSA) would be South East Asia, Europe, MENA, and East Africa depending on orbital slot.",
    "প্রাথমিক পরিসেবা এলাকা (পিএসএ) হবে দক্ষিন পূর্ব এশিয়া, ইউরোপ, মেনা এবং পূর্ব আফ্রিকার কক্ষপথের স্লট এর উপর নির্ভর করে। "
  ),
  (
    "The priority satellite applications are Direct to Home (DTH), VSAT, Backhaul and Trunking, Network Restoration, Disaster Preparedness and relief etc. ",
    "অগ্রাধিকার স্যাটেলাইট অ্যাপলিকেশনগুলি সরাসরি হোম (ডিথ), ভিএসএটি, ব্লাকহাউল এবং ট্রাঙ্কিং, নেটওয়ার্ক পুনরুদ্ধার,দুর্যোগ প্রস্তুতি ও ত্রান ইত্যাদি। "
  ),
  (
    "The researchers identified two genes in the greater mouse-eared bat that may be responsible for its unique longevity adaptation. ",
    "গবেষকরা দুটি জিন ইঁদুরের মত বড় কান অলা বাদুড়ে সনাক্ত করেন যা এর অনন্য দীর্ঘায়ু , অভিযোজনের কারণ হতে পারে। "
  ),
  (
    "The RF survey for probable sites has already been completed. ",
    "সম্ভাব্য সাইটের জন্য আরএফ জরিপ ইতিমধ্যেই সম্পন্ন হয়েছে।"
  ),
  (
    "The roads are all the same. ",
    "রাস্তাগুলি সব একই। "
  ),
  (
    "The S&P 500 ended another turbulent week on an upbeat note Friday. ",
    "এস এন্ড পি ৫০০ শুক্রবার আরো একটি উদ্বিগ্ন সপ্তাহ শেষ করলো। "
  ),
  (
    "The site of the university is in fact a pollution free area of Noakhali. ",
    "প্রকৃতপক্ষে নোয়াখালীতে বিশ্ববিদ্যালয়টির অবস্থানস্থল দূষণমুক্ত এলাকায়।"
  ),
  (
    "The Sundarbans contain the world's largest coastal mangrove forest, with an area of about 10,000 sq km (3,900 sq mi), of which about 6,000 sq km (2,300 sq mi) are located in Bangladesh.Sundarban is the part of bangladesh and india.",
    "সুন্দরবন বিশ্বের বৃহত্তম উপকূলীয় ম্যানগ্রোভ বন ধারণ করে, যার প্রায় 10,000 বর্গ কিমি (3,900 বর্গ মাইল) এলাকা রয়েছে, যার মধ্যে 6000 বর্গ কিলোমিটার (২300 বর্গ মাইল) বাংলাদেশে অবস্থিত।"
  ),
  (
    "The Sundarbans is a network of marine streams, mud shores and mangrove forests. ",
    "সুন্দরবন সামুদ্রিক নদী, কাদা সমুদ্র ও ম্যানগ্রোভ বনগুলির একটি নেটওয়ার্ক। "
  ),
  (
    "The Sundarbans is a vast forest,situated at the coastal region of the Bay of Bengal and considered one of the natural wonders of the world. ",
    "সুন্দরবন একটি বিশাল বন, বঙ্গোপসাগরের উপকূলবর্তী অঞ্চলে অবস্থিত এবং বিশ্বের প্রাকৃতিক বিস্ময়ের একটি হিসাবে বিবেচিত।"
  ),
  (
    "The system concept of the proposed satellite comprises with satellite payload requirements, orbital slot/frequencies, coverage area(s), ground segment, user terminal design characteristics, satellite operations and environmental factors. ",
    "প্রস্তাবিত উপগ্রহের সিস্টেম ধারনা উপগ্রহ প্লে লোড প্রয়োজনীয়তা কক্ষপথ স্লট বা ফ্রিকোয়ন্সি, কভারেজ এলাকাগুলি, গ্রাউন্ড সেগমেন্ট, ব্যাবহারকারী টারমিনাল নকশা বৈশিষ্ট, উপগ্রহ অপারেশন এবং পরিবেশগত কারনগুলির সঙ্গে রয়েছে।"
  ),
  (
    "The teachers are well educated having good art of presentation. ",
    "শিক্ষকগণ উচ্চশিক্ষিত এবং ভালভাবে উপস্থাপনের শিল্পগুন তাদের মধ্যে আছে।"
  ),
  (
    "The total estimated cost is 30 thousand 793 crore 39 lac taka.",
    "এতে আনুমানিক ব্যয় হচ্ছে ৩০ হাজার ৭৯৩ কোটি ৩৯ লাখ টাকা৷"
  ),
  (
    "The two-level steel truss bridge will carry a four-lane highway on the upper level and a single track railway on a lower level.",
    "দুই স্তর বিশিষ্ট ষ্টিল ট্রাস নির্মিত  ব্রিজটির ওপরের স্তরে থাকবে চার লেনের সড়ক পথ এবং নিচের স্তরটিতে থাকবে একটি একক রেলপথ। "
  ),
  (
    "The universe was once entirely dark, with nary a light anywhere throughout the entire cosmos.",
    "মহাবিশ্ব একসময় সম্পূর্ণ অন্ধকার ছিল, যেখানে সারা পৃথিবী জুড়ে সর্বদাই বিছিন্ন একটি আলো ছিল। "
  ),
  (
    "The work start on 7 December 2014. ",
    "কাজ শুরু হয় ৭ ডিসেম্বর ২০১৪। "
  ),
  (
    "Then another and another, leading to the stars and galaxies of the familiar universe. ",
    "তারপর এক এরপর এক বিস্ফোরণ, নেতৃত্ব দেয় পরিচিত মহাবিশ্বের তারা এবং ছায়াপথকে। "
  ),
  (
    "There are many reasons for it. ",
    "এর অনেক কারণ আছে। "
  ),
  (
    "There are many things I like to do, to see, and to experience. ",
    "অনেক কিছু আছে যা আমি করতে, দেখতে  এবং অভিজ্ঞতা নিতে পছন্দ করি । "
  ),
  (
    "There are many unlicensed vehicles which should be brought under control. ",
    "অনেক অননুমোদিত যানবাহন রয়েছে যা নিয়ন্ত্রণে আনা উচিত। "
  ),
  (
    "There will be two ground stations for satellite operation and control, one as the primary site and other as the backup site. ",
    "উপগ্রহ অপারেশন এবং নিয়ন্ত্রন এর জন্য ২টি স্থল স্টেশন হবে, প্রাথমিক সাইট হিসেবে এবং ব্যাকআপ সাইট হিসেবে অন্য একটি। "
  ),
  (
    "These are, however, very much linked with the overall nation building process which is manifested in the Constitution as the Fundamental principles of State policy. ",
    "তবে, সামগ্রিক জাতি গঠনের প্রক্রিয়ার সাথে এগুলি  খুব বেশী সম্পর্কিত, যা রাষ্ট্রীয় নীতির মৌলিক তত্ব হিসাবে সংবিধানে প্রকাশ পায়। "
  ),
  (
    "These mechanisms could be the focus of future studies on aging, with an eye toward extending healthy lifespans in people, the researchers said.",
    "এসব প্রক্রিয়া মানুষের বার্ধক্যকে সুস্থ জীবনকালের সাথে সম্প্রসারন করে ভবিষ্যৎ গবেষণার কেন্দ্র হতে পারে, গবেষকরা বলেন।"
  ),
  (
    "They are also very much amicable.",
    "তারা যথেষ্ট বন্ধুসুলভ ও।"
  ),
  (
    "They are characterized by erect posture and bipedal locomotion; high manual dexterity and heavy tool use compared to other animals; and a general trend toward larger, more complex brains and societies.",
    "তাদের হল বৈশিষ্ট দ্বারা স্থির অবস্থান  এবং গতিশক্তি; উচ্চ দক্ষতাসম্পন্ন  এবং ভারী সরঞ্জাম ব্যবহার তুলনা থেকে অন্যান্য প্রাণীর; এবং একটি সাধারন ঝোক অভিমুখে বৃহত্তর, আরো জটিল মস্তিষ্ক এবং সমাজ ।"
  ),
  (
    "They want to drive at their sweet will. ",
    "তারা তাদের ইচ্ছামত চালাতে চান। "
  ),
  (
    "This city is located on the banks of the Karnaphuli River between the Chittagong Hill Tracts and the Bay of Bengal. ",
    "এই শহর পার্বত্য চট্টগ্রাম ও বঙ্গোপসাগরের মধ্যে কর্ণফুলী নদীর তীরে অবস্থিত। "
  ),
  (
    "This drives the natural aging process, leading to a breakdown of cells that over time can drive tissue deterioration and eventually death.",
    "এটা প্রতিবার প্রাকৃতিক বার্ধক্য প্রক্রিয়াকে কোষ মৃত্যুর দিকে ধাবিত করে। যা নির্দিষ্ট সময়ের পরে টিস্যু ক্ষয় এবং অবশেষে মৃত্যুর দিকে ঠেলে দেয়।"
  ),
  (
    "This followed a bilateral meeting where they discussed the bilateral issues, the foreign ministry said. ",
    "পররাষ্ট্র মন্ত্রণালয় বলেছে একটি দ্বিপাক্ষিক বৈঠকে দ্বিপাক্ষিক বিষয় নিয়ে আলোচনার পর এটা হয়েছে । "
  ),
  (
    "This is a lot easier for those who were trying to use Deep Learning AI.",
    "যাঁরা ডিপ লার্নিং এআই ব্যবহার করার ব্যাপারে চেষ্টা করছিলেন, তাঁদের জন্য বিষয়টি অনেকটাই সহজ হবে|"
  ),
  (
    "This makes Chittagong the crossroads of the Indian subcontinent and Southeast Asia.",
    "এটি ভারতের উপমহাদেশ এবং দক্ষিণ-পূর্ব এশিয়ায় ক্রস রোড নির্মাণ করে।"
  ),
  (
    "This slightly acidic level prevents fungal and bacterial growth in hair and scalp, and keeps the cuticles closed and healthy.",
    "সামান্য আম্লিক মাত্রা চুলে ছত্রাক এবং ব্যাকটেরিয়া বৃদ্ধি রোধে সহায়াতা করে, কিউটিকল গুলা বন্ধ ও স্বাস্থ্যকর রাখে। "
  ),
  (
    "This unique forest extends across Khulna, Satkhira, Bagerhat districts of Bangladesh.",
    "এই অনন্য বন বাংলাদেশের খুলনা, সাতক্ষীরা, বাগেরহাট জেলার বিস্তৃত।"
  ),
  (
    "Though skipper Shakib has been included in the squad but there is still a doubt if he can play the match. ",
    "যদিও স্পিনার সাকিবকে অন্তর্ভুক্ত করা হয়েছে কিন্তু সে খেলতে পারবে কিনা এখনও সন্দেহ আছে। "
  ),
  (
    "Traffic jam is a common affair in the big cities and towns. ",
    "ট্র্যাফিক জ্যাম বড় শহরে এবং শহরে একটি সাধারণ ব্যাপার। "
  ),
  (
    "Two have died in a powerful earthquake in Taiwan. ",
    "তাইওয়ানে শক্তিশালী ভূমিকম্পে দুজনের মৃত্যু হয়েছে। "
  ),
  (
    "US President Donald Trump struck a defiant tone on Friday, saying trade wars were good and easy to win, after his plan to put tariffs on steel and aluminium imports triggered threats of retaliation from trading partners and a slide in stock markets. ",
    "ইস্পাত এবং অ্যালুমিনিয়াম আমদানিতে ট্যারিফের পরিকল্পনা করার পর বাণিজ্য অংশীদারদের কাছ থেকে পাল্টা বেবস্থার হুমকি সৃষ্টি এবং স্টক মার্কেটে ধস নামার পর, যুক্তরাষ্ট্রের প্রেসিডেন্ট ডোনাল্ড ট্রাম্প শুক্রবার উদ্ধত স্বরে বলেন “বাণিজ্য যুদ্ধ ভাল এবং এতে জয় লাভ করা সহজ”।   "
  ),
  (
    "Usually, this means that you are using a product that has a high pH above 7, and it is stripping away your naturally acidic sebum that fights the microbial growth.",
    "সাধারণত এটা বোঝায়ে যে আপনি এমন প্রসাধনী ব্যবহার করছেন যার pH  ৭ এর উপরে, এবং  এটা আপনার প্রাকিতিক ভাবে তৈরি আম্লিও  সেবাম মুছে দিচ্ছে যা অণুজীব বৃদ্ধি  সাথে লড়াই করে। "
  ),
  (
    "Water is a transparent and nearly colorless chemical substance that is the main constituent of Earth's streams, lakes, and oceans, and the fluids of most living organisms.",
    "জল একটি স্বচ্ছ এবং প্রায় বর্ণহীন রাসায়নিক পদার্থ যা পৃথিবীর প্রবাহ, হ্রদ, এবং মহাসাগরের প্রধান উপাদান, এবং অধিকাংশ জীবন্ত প্রাণীর তরল।"
  ),
  (
    "We must be aware of developing international relation to achieve the target. ",
    "এই স্বপ্ন বাস্তবায়নে আন্তর্জাতিক সম্পর্কের উন্নয়নে অবশ্যই সচেতন থাকতে হবে। "
  ),
  (
    "While the ideal pH level of your skin remains a steady 5.5, hair has two separate numbers, the ideal pH for your actual hair strands is 3.67, but for the scalp, it's 5.5.",
    "যদিও আপনার ত্বক এর আদর্শ pH ৫.৫ এ স্থির থাকে, কিন্তু চুলের জন্য দুইটি ভিন্ন  সংখ্যা আছে, চুলের আদর্শ pH 3.67 এবং স্কাল্পের ৫.৫।  "
  ),
  (
    "With 150 span, 6150 m total length and 18.10 m width it is going to be the largest bridge in the Padma-Brahmaputra-Meghna river basins of the country in terms of both span and the total length.",
    "পদ্মা-ব্রহ্মপুত্র-মেঘনা নদীর আববাহিকায় ১৫০টি স্পান, সর্বমোট ৬,১৫০ মিটার দৈর্ঘ্য এবং ১৮.১০ মিটার প্রস্থ, উভয় স্প্যান এবং মোট দৈর্ঘ্যের পরিপ্রেক্ষিতে নির্মিত হচ্ছে দেশটির সবচে বড় সেতু।"
  ),
  (
    "With the help of artificial intelligence, by this new system each device can use the internet independently; which is much like the neural network. ",
    "কৃত্রিম বুদ্ধিমত্তার সাহায্যে নতুন এই পদ্ধতির মাধ্যমে প্রতিটি যন্ত্র স্বাধীনভাবে ইন্টারনেট ব্যবহার করতে পারবে; যা অনেকটাই নিউরাল নেটওয়ার্কের মতো |"
  )


    
)

In [35]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def normalize_english(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.,?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

def normalize_bangla(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!।,?])', r' \1', s)
    s = re.sub(r'\s+', r' ', s)
    return s




In [39]:
raw_data_input, raw_data_output = zip(*raw_data)
raw_data_input, raw_data_output = list(raw_data_input), list(raw_data_output)
raw_data_input = [normalize_english(data) for data in raw_data_input]
raw_data_output_in = ['<start> ' + normalize_bangla(data) for data in raw_data_output]
raw_data_output_out = [normalize_bangla(data) + ' <end>' for data in raw_data_output]


In [6]:
input_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
input_tokenizer.fit_on_texts(raw_data_input)
data_en = en_tokenizer.texts_to_sequences(raw_data_en)
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en,
                                                        padding='post')

fr_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
fr_tokenizer.fit_on_texts(raw_data_fr_in)
fr_tokenizer.fit_on_texts(raw_data_fr_out)
data_fr_in = fr_tokenizer.texts_to_sequences(raw_data_fr_in)
data_fr_in = tf.keras.preprocessing.sequence.pad_sequences(data_fr_in,
                                                           padding='post')

data_fr_out = fr_tokenizer.texts_to_sequences(raw_data_fr_out)
data_fr_out = tf.keras.preprocessing.sequence.pad_sequences(data_fr_out,
                                                            padding='post')

In [7]:
BATCH_SIZE = 5
dataset = tf.data.Dataset.from_tensor_slices(
    (data_en, data_fr_in, data_fr_out))
dataset = dataset.shuffle(20).batch(BATCH_SIZE)

In [8]:
def positional_embedding(pos, model_size):
    PE = np.zeros((1, model_size))
    for i in range(model_size):
        if i % 2 == 0:
            PE[:, i] = np.sin(pos / 10000 ** (i / model_size))
        else:
            PE[:, i] = np.cos(pos / 10000 ** ((i - 1) / model_size))
    return PE

max_length = max(len(data_en[0]), len(data_fr_in[0]))
MODEL_SIZE = 128

pes = []
for i in range(max_length):
    pes.append(positional_embedding(i, MODEL_SIZE))

pes = np.concatenate(pes, axis=0)
pes = tf.constant(pes, dtype=tf.float32)

In [9]:
class MultiHeadAttention(tf.keras.Model):
    def __init__(self, model_size, h):
        super(MultiHeadAttention, self).__init__()
        self.query_size = model_size // h
        self.key_size = model_size // h
        self.value_size = model_size // h
        self.h = h
        self.wq = [tf.keras.layers.Dense(self.query_size) for _ in range(h)]
        self.wk = [tf.keras.layers.Dense(self.key_size) for _ in range(h)]
        self.wv = [tf.keras.layers.Dense(self.value_size) for _ in range(h)]
        self.wo = tf.keras.layers.Dense(model_size)

    def call(self, decoder_output, encoder_output):
        # decoder_output has shape (batch, decoder_len, model_size)
        # encoder_output has shape (batch, encoder_len, model_size)
        heads = []
        for i in range(self.h):
            score = tf.matmul(self.wq[i](decoder_output), self.wk[i](encoder_output), transpose_b=True) / tf.math.sqrt(tf.dtypes.cast(self.key_size, tf.float32))
            # score has shape (batch, decoder_len, encoder_len)
            alignment = tf.nn.softmax(score, axis=2)
            # alignment has shape (batch, decoder_len, encoder_len)
            head = tf.matmul(alignment, self.wv[i](encoder_output))
            # head has shape (batch, decoder_len, value_size)
            heads.append(head)
        heads = tf.concat(heads, axis=2)
        heads = self.wo(heads)
        # heads has shape (batch, decoder_len, model_size)
        return heads

In [10]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, model_size, num_layers, h):
        super(Encoder, self).__init__()
        self.model_size = model_size
        self.num_layers = num_layers
        self.h = h
        self.embedding = tf.keras.layers.Embedding(vocab_size, model_size)
        self.attention = [MultiHeadAttention(model_size, h) for _ in range(num_layers)]
        
        self.attention_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
        self.dense_1 = [tf.keras.layers.Dense(512, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(model_size) for _ in range(num_layers)]
        self.ffn_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
    def call(self, sequence):
        sub_in = []
        for i in range(sequence.shape[1]):
            embed = self.embedding(tf.expand_dims(sequence[:, i], axis=1))
            sub_in.append(embed + pes[i, :])
            
        sub_in = tf.concat(sub_in, axis=1)
        
        for i in range(self.num_layers):
            sub_out = []
            for j in range(sub_in.shape[1]):
                attention = self.attention[i](
                    tf.expand_dims(sub_in[:, j, :], axis=1), sub_in)

                sub_out.append(attention)

            sub_out = tf.concat(sub_out, axis=1)
            sub_out = sub_in + sub_out
            sub_out = self.attention_norm[i](sub_out)
            
            ffn_in = sub_out

            ffn_out = self.dense_2[i](self.dense_1[i](ffn_in))
            ffn_out = ffn_in + ffn_out
            ffn_out = self.ffn_norm[i](ffn_out)

            sub_in = ffn_out
            
        return ffn_out

In [11]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, model_size, num_layers, h):
        super(Decoder, self).__init__()
        self.model_size = model_size
        self.num_layers = num_layers
        self.h = h
        self.embedding = tf.keras.layers.Embedding(vocab_size, model_size)
        self.attention_bot = [MultiHeadAttention(model_size, h) for _ in range(num_layers)]
        self.attention_bot_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.attention_mid = [MultiHeadAttention(model_size, h) for _ in range(num_layers)]
        self.attention_mid_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
        self.dense_1 = [tf.keras.layers.Dense(512, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(model_size) for _ in range(num_layers)]
        self.ffn_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, sequence, encoder_output):
        # EMBEDDING AND POSITIONAL EMBEDDING
        embed_out = []
        for i in range(sequence.shape[1]):
            embed = self.embedding(tf.expand_dims(sequence[:, i], axis=1))
            embed_out.append(embed + pes[i, :])
            
        embed_out = tf.concat(embed_out, axis=1)
        
        
        bot_sub_in = embed_out
        
        for i in range(self.num_layers):
            # BOTTOM MULTIHEAD SUB LAYER
            bot_sub_out = []
            
            for j in range(bot_sub_in.shape[1]):
                values = bot_sub_in[:, :j, :]
                attention = self.attention_bot[i](
                    tf.expand_dims(bot_sub_in[:, j, :], axis=1), values)

                bot_sub_out.append(attention)
            bot_sub_out = tf.concat(bot_sub_out, axis=1)
            bot_sub_out = bot_sub_in + bot_sub_out
            bot_sub_out = self.attention_bot_norm[i](bot_sub_out)
            
            # MIDDLE MULTIHEAD SUB LAYER
            mid_sub_in = bot_sub_out

            mid_sub_out = []
            for j in range(mid_sub_in.shape[1]):
                attention = self.attention_mid[i](
                    tf.expand_dims(mid_sub_in[:, j, :], axis=1), encoder_output)

                mid_sub_out.append(attention)

            mid_sub_out = tf.concat(mid_sub_out, axis=1)
            mid_sub_out = mid_sub_out + mid_sub_in
            mid_sub_out = self.attention_mid_norm[i](mid_sub_out)

            # FFN
            ffn_in = mid_sub_out

            ffn_out = self.dense_2[i](self.dense_1[i](ffn_in))
            ffn_out = ffn_out + ffn_in
            ffn_out = self.ffn_norm[i](ffn_out)

            bot_sub_in = ffn_out
        
        logits = self.dense(ffn_out)
            
        return logits

In [12]:
H = 2
NUM_LAYERS = 2

en_vocab_size = len(en_tokenizer.word_index) + 1
encoder = Encoder(en_vocab_size, MODEL_SIZE, NUM_LAYERS, H)

en_sequence_in = tf.constant([[1, 2, 3, 4, 6, 7, 8, 0, 0, 0], 
                           [1, 2, 3, 4, 6, 7, 8, 0, 0, 0]])
encoder_output = encoder(en_sequence_in)

print('Input vocabulary size', en_vocab_size)
print('Encoder input shape', en_sequence_in.shape)
print('Encoder output shape', encoder_output.shape)

fr_vocab_size = len(fr_tokenizer.word_index) + 1
max_len_fr = data_fr_in.shape[1]
decoder = Decoder(fr_vocab_size, MODEL_SIZE, NUM_LAYERS, H)

fr_sequence_in = tf.constant([[1, 2, 3, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0],
                           [1, 2, 3, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0]])
decoder_output = decoder(fr_sequence_in, encoder_output)

print('Target vocabulary size', fr_vocab_size)
print('Decoder input shape', fr_sequence_in.shape)
print('Decoder output shape', decoder_output.shape)

Input vocabulary size 1262
Encoder input shape (2, 10)
Encoder output shape (2, 10, 128)
Target vocabulary size 11
Decoder input shape (2, 14)
Decoder output shape (2, 14, 11)


In [13]:
crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True)
def loss_func(targets, logits):
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss


optimizer = tf.keras.optimizers.Adam()

In [14]:
def predict(test_source_text=None):
    if test_source_text is None:
        test_source_text = raw_data_en[np.random.choice(len(raw_data_en))]
    print(test_source_text)
    test_source_seq = en_tokenizer.texts_to_sequences([test_source_text])
    print(test_source_seq)

    en_output = encoder(tf.constant(test_source_seq))

    de_input = tf.constant([[fr_tokenizer.word_index['<start>']]], dtype=tf.int64)

    out_words = []

    while True:
        de_output = decoder(de_input, en_output)
        new_word = tf.expand_dims(tf.argmax(de_output, -1)[:, -1], axis=1)
        out_words.append(fr_tokenizer.index_word[new_word.numpy()[0][0]])

        de_input = tf.concat((de_input, new_word), axis=-1)

        if out_words[-1] == '<end>' or len(out_words) >= 14:
            break

    print(' '.join(out_words))

In [15]:

def train_step(source_seq, target_seq_in, target_seq_out):
    with tf.GradientTape() as tape:
        encoder_output = encoder(source_seq)
        
        decoder_output = decoder(target_seq_in, encoder_output)

        loss = loss_func(target_seq_out, decoder_output)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [ ]:
NUM_EPOCHS = 100

start_time = time.time()
for e in range(NUM_EPOCHS):
    for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
        loss = train_step(source_seq, target_seq_in,
                          target_seq_out)

    print('Epoch {} Loss {:.4f}'.format(
          e + 1, loss.numpy()))

    if (e + 1) % 10 == 0:
        end_time = time.time()
        print('Average elapsed time: {:.2f}s'.format((end_time - start_time) / (e + 1)))
        try:
            predict()
        except Exception as e:
            print(e)
            continue

Epoch 1 Loss 36.5495
Epoch 2 Loss 8.8123
Epoch 3 Loss 16.3901
Epoch 4 Loss 3.5839
Epoch 5 Loss 4.5305
Epoch 6 Loss 6.6410
Epoch 7 Loss 2.6816
Epoch 8 Loss 3.0848
Epoch 9 Loss 2.4130
Epoch 10 Loss 3.5546
Average elapsed time: 91.30s
During the final of the tri nation series against Sri Lanka all rounder Shakib Al Hasan injured his finger while fielding and it didn t only ruled him out from that match but also for the two match Test series .
[[250, 1, 578, 3, 1, 579, 83, 129, 580, 581, 582, 40, 583, 97, 584, 585, 251, 148, 586, 149, 587, 4, 16, 588, 131, 80, 589, 590, 145, 47, 10, 62, 64, 103, 12, 1, 36, 62, 98, 129, 2]]
. . . . . . . . . . . . . .
Epoch 11 Loss 1.5429
Epoch 12 Loss 1.5000
Epoch 13 Loss 1.8798
Epoch 14 Loss 2.5161
Epoch 15 Loss 2.9978
Epoch 16 Loss 1.9079
Epoch 17 Loss 2.2822
Epoch 18 Loss 2.8352
Epoch 19 Loss 1.5793
Epoch 20 Loss 1.4481
Average elapsed time: 90.88s
Traffic jam is a common affair in the big cities and towns . 
[[46, 95, 8, 6, 1200, 1201, 7, 1, 1202, 143,

In [ ]:
test_sents = (
    "A number of universities are there in Bangladesh.",
    "Two have died in a powerful earthquake in Taiwan. "
)

for test_sent in test_sents:
    test_sequence = normalize_string(test_sent)
    predict(test_sequence)